# Outline
- clean text
- tf-idf text for weights
- embeddings
    - word2vec words * tf-idf
    - save to each row of each df

In [1]:
import time
import pickle
import pandas as pd

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # for part of speech tagging, required for lemmatization
nltk.download('stopwords')
from nltk.corpus import stopwords



import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
# import gensim.sklearn_api
import gensim.models.phrases


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df.pkl'

with open(DATA_PATH + DF_NAME, 'rb') as f:
    df = pickle.load(f)


In [3]:
df.head()

,created_at,user,id_str,retweeted_status,Hash words,link,entities,Topic Label,urls,no_link,...,domains,day_of_tweet,month_year,week_month_year,user_id,cluster,is_retweet,hashtags,link_shorteners,article_text
0,2020-01-31 23:08:33+00:00,"{'id': 19031057, 'id_str': '19031057', 'name':...",1223382589689356288,NaN,#coronavirus,https://twitter.com/user/status/12233825896893...,"{'hashtags': [{'text': 'coronavirus', 'indices...","(43, 0.999852366207409)",[https://www.kron4.com/news/national/when-will...,False,...,[www.kron4.com],2020-01-31,2020-01,5,19031057,43,False,[coronavirus],[False],"KRON4 by: Aubree Gordon, University of Michig..."
1,2020-01-31 23:11:29+00:00,"{'id': 798925214, 'id_str': '798925214', 'name...",1223383328843280384,{'created_at': 'Fri Jan 31 20:41:20 +0000 2020...,#coronavirus #vaccine:,https://twitter.com/user/status/12233833288432...,"{'hashtags': [{'text': 'Coronavirus', 'indices...","(6, 0.9883827140131066)",[https://www.cnbc.com/2020/01/31/coronavirus-w...,False,...,[www.cnbc.com],2020-01-31,2020-01,5,798925214,6,True,"[Coronavirus, vaccine]",[False],U.S. and international health officials are s...
2,2020-01-31 23:12:03+00:00,"{'id': 806144538049970176, 'id_str': '80614453...",1223383471999127552,NaN,No hashtags,https://twitter.com/user/status/12233834719991...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999928079220538)",[https://www.businessinsider.com/australia-suc...,False,...,[www.businessinsider.com],2020-01-31,2020-01,5,806144538049970176,74,False,[None],[False],A leading-edge research firm focused on digit...
3,2020-01-31 23:35:42+00:00,"{'id': 61298849, 'id_str': '61298849', 'name':...",1223389423045206016,{'created_at': 'Tue Jan 28 18:26:16 +0000 2020...,No hashtags,https://twitter.com/user/status/12233894230452...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(36, 0.9999993854913211)",[https://www.greenmedinfo.com/blog/examining-r...,False,...,[www.greenmedinfo.com],2020-01-31,2020-01,5,61298849,36,True,[None],[False],Mr. Kennedy is in very safe territory by rep...
4,2020-01-31 23:37:23+00:00,"{'id': 1152822375567654912, 'id_str': '1152822...",1223389844719599616,NaN,No hashtags,https://twitter.com/user/status/12233898447195...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999985797901156)",[https://www.dailymail.co.uk/news/article-7952...,False,...,[www.dailymail.co.uk],2020-01-31,2020-01,5,1152822375567654912,74,False,[None],[True],By Kylie Stevens and Stephen Gibbs and Nic Wh...


In [4]:
# def sent_to_words(sentences):
#     """Gensim simple preprocess. Turns each element in list of strings to a list of tokens. removes punct"""
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def simple_tokenize(texts):
    doc_toks = []
    for i in texts:
        doc_toks.append(list(gensim.utils.tokenize(i, deacc = True, lowercase=True))) # where i is a string
    return doc_toks


def remove_stopwords_and_simple_preprocess(texts):
    stop_words = stopwords.words('english')
#     common = []
#     with open('./data/raw/common_words.csv', 'r') as f:
#         reader = csv.reader(f)
#         for line in reader:
#             common.extend(line)
#     stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'employ', 'benefit'])
#     stop_words.extend(common)
    return [[word for word in simple_preprocess(str(doc), deacc = True) if word not in stop_words] for doc in texts]


# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]


# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]



In [5]:
# # NLTK Lemmatize with POS Tag
# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}

#     return tag_dict.get(tag, wordnet.NOUN)


# def lemmatization(texts): # texts is list of lists
#     t0 = time.time()
#     print('lemmatizing...')
#     lemmatizer = WordNetLemmatizer()
#     texts_out = []
#     for doc in texts: # the document is a list
#         texts_out.append([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in doc])
#     print('lemmatization complete')
#     print('{} seconds'.format(time.time()-t0))
#     return texts_out


In [8]:
def full_clean(data):
    """data is a list of strings, uncleaned, each element a document"""

    # tokenize, lowercase, remove punct, keep common words for bigram building
    t0 = time.time()
    print('simple tokenizing...')
    data_words = simple_tokenize(data)
    print('simple_tokenizing took {} seconds'.format(time.time()-t0))
    
   
    ### Build bigram model
    t0 = time.time()
    print('building bigrams...')
    common_connectors = ["of", "with", "without", "and", "or", "the", "a"]
    bigram = gensim.models.Phrases(data_words, 
                                   min_count=25, 
                                   threshold=100, 
#                                    connector_words=common_connectors) # let common eng articles connect words
                                   common_terms = common_connectors)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_words_bigrams = [bigram_mod[doc] for doc in data_words]
    print('building bigrams took {} seconds'.format(time.time()-t0))

    ### Build trigram model
    t0 = time.time()
    print('building trigrams...')
    trigram = gensim.models.Phrases(bigram[data_words], 
                                    min_count=25, 
                                    threshold=100, 
#                                     connector_words = common_connectors)
                                    common_terms = common_connectors)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    data_words_trigrams = [trigram_mod[bigram_mod[doc]] for doc in data_words_bigrams]
    print('building trigrams took {} seconds'.format(time.time()-t0))
    
    
    # remake into strings
    texts = []
    for i in data_words_trigrams:
        string = ' '.join(i)
        texts.append(string)
    
    
    # Remove Stop Words
    t0 = time.time()
    print('remove_stopwords_and_simple_preprocess...')
    data_words_trigrams_nostops = remove_stopwords_and_simple_preprocess(texts)
    print('remove_stopwords_and_simple_preprocess took {} seconds'.format(time.time()-t0))

    # return a lemmatized list of unigram + bigram + trigrams
#     return lemmatization(data_words_trigrams)
    return data_words_trigrams_nostops

In [9]:
df['article_text_cleaned'] = full_clean(df['article_text'].tolist())

# simple_tokenizing took 50.34017825126648 seconds
# building bigrams took 88.33401918411255 seconds
# building trigrams took 192.49179601669312 seconds
# remove_stopwords_and_simple_preprocess took 70.06075119972229 seconds

simple tokenizing...
simple_tokenizing took 50.34017825126648 seconds
building bigrams...
building bigrams took 88.33401918411255 seconds
building trigrams...
building trigrams took 192.49179601669312 seconds
remove_stopwords_and_simple_preprocess...
remove_stopwords_and_simple_preprocess took 70.06075119972229 seconds


In [13]:
df.head()

,created_at,user,id_str,retweeted_status,Hash words,link,entities,Topic Label,urls,no_link,...,day_of_tweet,month_year,week_month_year,user_id,cluster,is_retweet,hashtags,link_shorteners,article_text,article_text_cleaned
0,2020-01-31 23:08:33+00:00,"{'id': 19031057, 'id_str': '19031057', 'name':...",1223382589689356288,NaN,#coronavirus,https://twitter.com/user/status/12233825896893...,"{'hashtags': [{'text': 'coronavirus', 'indices...","(43, 0.999852366207409)",[https://www.kron4.com/news/national/when-will...,False,...,2020-01-31,2020-01,5,19031057,43,False,[coronavirus],[False],"KRON4 by: Aubree Gordon, University of Michig...","[kron, aubree, gordon, university, michigan, c..."
1,2020-01-31 23:11:29+00:00,"{'id': 798925214, 'id_str': '798925214', 'name...",1223383328843280384,{'created_at': 'Fri Jan 31 20:41:20 +0000 2020...,#coronavirus #vaccine:,https://twitter.com/user/status/12233833288432...,"{'hashtags': [{'text': 'Coronavirus', 'indices...","(6, 0.9883827140131066)",[https://www.cnbc.com/2020/01/31/coronavirus-w...,False,...,2020-01-31,2020-01,5,798925214,6,True,"[Coronavirus, vaccine]",[False],U.S. and international health officials are s...,"[international, health, officials, speeding, w..."
2,2020-01-31 23:12:03+00:00,"{'id': 806144538049970176, 'id_str': '80614453...",1223383471999127552,NaN,No hashtags,https://twitter.com/user/status/12233834719991...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999928079220538)",[https://www.businessinsider.com/australia-suc...,False,...,2020-01-31,2020-01,5,806144538049970176,74,False,[None],[False],A leading-edge research firm focused on digit...,"[leading_edge, research, firm_focused, active,..."
3,2020-01-31 23:35:42+00:00,"{'id': 61298849, 'id_str': '61298849', 'name':...",1223389423045206016,{'created_at': 'Tue Jan 28 18:26:16 +0000 2020...,No hashtags,https://twitter.com/user/status/12233894230452...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(36, 0.9999993854913211)",[https://www.greenmedinfo.com/blog/examining-r...,False,...,2020-01-31,2020-01,5,61298849,36,True,[None],[False],Mr. Kennedy is in very safe territory by rep...,"[mr, kennedy, safe, territory, reporting, cdc,..."
4,2020-01-31 23:37:23+00:00,"{'id': 1152822375567654912, 'id_str': '1152822...",1223389844719599616,NaN,No hashtags,https://twitter.com/user/status/12233898447195...,"{'hashtags': [], 'symbols': [], 'user_mentions...","(74, 0.9999985797901156)",[https://www.dailymail.co.uk/news/article-7952...,False,...,2020-01-31,2020-01,5,1152822375567654912,74,False,[None],[True],By Kylie Stevens and Stephen Gibbs and Nic Wh...,"[kylie, stevens, stephen, gibbs, nic, white, d..."


In [15]:
def make_dict(texts):
    """Takes a list of lists which contain strings (lemmatized/tokenized)"""
    """This dictionary is often referred to as id2word in NLP posts"""
    # uses gensim.corpora.Dictionary
    return corpora.Dictionary(texts)

def make_bow_corpus(texts, dictionary):
    """Creates bag of words model"""
    """texts is a list of lists; doc is a list of strings"""
    """dictionary made using make_dict"""
    # uses gensim
    return [dictionary.doc2bow(doc) for doc in texts]

In [16]:
articles_dict = make_dict(df['article_text_cleaned'].tolist())
articles_corpus = make_bow_corpus(df['article_text_cleaned'].tolist(), articles_dict)


In [17]:
### save df
DATA_PATH = './data/cleaned/'
DF_NAME = 'concatenated_df_cleaned.pkl'

with open(DATA_PATH + DF_NAME, 'wb') as f:
    pickle.dump(df, f)



In [18]:
### save dictionary and bow_corpus
articles_dict_filename = 'articles_dict.pkl'
with open(DATA_PATH + articles_dict_filename, 'wb') as f:
    pickle.dump(articles_dict, f)

articles_corpus_filename = 'articles_corpus.pkl'
with open(DATA_PATH + articles_corpus_filename, 'wb') as f:
    pickle.dump(articles_corpus, f)


# copied from 'make_doc2vec_model.py' in monster directory

In [ ]:
# create doc2vec model of monster jobs to calculate similarity

### imports
import pandas as pd
import pickle
# import numpy as np
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
# from gensim.utils import simple_preprocess

# from nltk.corpus import stopwords
# STOPWORDS = stopwords.words('english')



class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])



### load data
CLEAN_DATA_PATH = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'
# load cleaned text
with open(CLEAN_DATA_PATH + 'monster_jobs_cleaned_text.pkl', 'rb') as f:
    cleaned_text = pickle.load(f)

# load df
with open(CLEAN_DATA_PATH + 'monster_jobs_df_small.pkl', 'rb') as f:
    df = pickle.load(f)
df['cleaned_description'] = cleaned_text
docs = list(df['cleaned_description'])
labels = list(df['id'])

sentences = TaggedDocumentIterator(docs, labels)


# create doc2vec model
model = Doc2Vec(vector_size=100,
                 window=5,
                 min_count=20,
                 workers=1,
                 alpha=0.025,
#                  min_alpha=0.0025,
                 epochs=100,
                 dm=0,        #1 = paragraph vector - distributed memory; 0 = dbow
                 seed=42)

# train model
model.build_vocab(sentences)
model.train(sentences,
             total_examples=model.corpus_count,
             epochs=model.epochs)

print('created Doc2Vec model')
print('Distributed Mem (True) or BOW (false): {}'.format(model.dm))
print('vector length: {}'.format(model.vector_size))
print('corpus count: {}'.format(model.corpus_count))
print('epochs: {}'.format(model.epochs))


print('saving d2v model...')
MODEL_PATH = '/Users/richardkuzma/coding/analysis/monster/models/'
model_name = 'd2v_' + 'dm_' + str(int(model.dm)) +'_vecsize_' + str(model.vector_size) + '_epochs_' + str(model.epochs) + '.model'
model.save(MODEL_PATH + model_name)
print('saved d2v model. \n location: ' + MODEL_PATH + '\nname: ' + model_name)
